In [1]:
import sys
import os
import argparse
import json

# 添加项目路径
sys.path.append('/Users/haha/Story')  # 你的项目根目录

# 导入所有必要模块
from src.constant import output_dir
from src.utils.utils import save_md, save_json, load_json, extract_plot_list, generate_response, convert_json
from src.generation.outline_generator import generate_outline
from src.generation.chapter_reorder import reorder_chapters
from src.generation.generate_characters import generate_characters_v1
from src.generation.expand_story import expand_story_v1
from src.compile_story import compile_full_story_by_sentence, compile_full_story_by_chapter
from src.enhance_story import enhance_story_with_transitions, polish_dialogues_in_story
from src.generation.dialogue_inserter import analyze_dialogue_insertions, run_dialogue_insertion, analyze_dialogue_insertions_v2
from src.utils.utils import extract_behavior_llm, convert_dialogue_dict_to_list
from src.sync.plot_sync_manager import sync_plot_and_dialogue_from_behavior
from src.sync.auto_propagate_plot_update import auto_propagate_plot_update
from src.analysis.character_state_tracker import run_character_state_tracker
from src.utils.logger import append_log, build_log_record, build_simple_log, init_log_path
from src.version_namer import build_version_name 

print("✅ 所有模块导入成功")

#

✅ 所有模块导入成功


In [2]:
# 设置项目路径
project_path = "/Users/haha/Story"
sys.path.append(project_path)

# 导入必要的函数
try:
    from src.utils.utils import generate_response, convert_json
    print("✅ 成功导入项目模块")
except ImportError as e:
    print(f"⚠️ 导入失败: {e}")
    # 模拟函数
    def generate_response(messages):
        return '{"next_speaker": "NONE"}'
    def convert_json(text):
        try:
            return json.loads(text)
        except:
            return {}

# 测试数据
test_sentence_context = "小红帽刚踏出飞船舱门,星际森林的低语与机械残骸的回声交织在耳边"
test_candidates = ["小红帽", "赛博狼"]
test_characters = [
    {"name": "小红帽", "description": "勇敢的快递员"},
    {"name": "赛博狼", "description": "危险的黑客"}
]

print("🧪 测试环境准备完成")

✅ 成功导入项目模块
🧪 测试环境准备完成


In [3]:
# Cell 2: 原版函数（从你的代码复制）
def generate_dialogue_for_insertion_original(sentence_context, candidate_characters, full_plot, character_personality):
    """原版函数，用于对比"""
    print(f"\n🔍 原版：开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""
    MAX_ROUNDS = 5  # 测试时降低轮数

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，请基于以下剧情做出第一句发言。
剧情背景是：{sentence_context}
你可以向其他角色说话：{[c for c in candidate_characters if c != speaker]}
用以下json格式返回：
{{"dialogue": "...", "action": "..."}}"""
    }]
    
    response = generate_response(prompt_first)
    parsed = convert_json(response)
    
    if isinstance(parsed, dict) and "dialogue" in parsed:
        spoken_line = parsed.get("dialogue", "")
        action = parsed.get("action", "")
        dialogue_list.append({
            "speaker": speaker,
            "dialogue": spoken_line,
            "action": action
        })
        history += f"{speaker}: {spoken_line}\n"

    # 多轮对话循环
    round_count = 0
    while round_count < MAX_ROUNDS:
        round_count += 1
        print(f"\n  🔄 原版第{round_count}轮对话")
        
        # 原版判断逻辑
        speaker_prompt = [{
            "role": "system",
            "content": f"""你是故事编剧。
当前剧情：{sentence_context}
当前已有对话历史：
{history}

只能从以下角色中选择发言：{candidate_characters}
请判断下一位发言人是谁？如果不需要继续对话，返回"NONE"。
格式：{{"next_speaker": "角色"}}"""
        }]
        
        next_res = generate_response(speaker_prompt)
        next_data = convert_json(next_res)
        next_speaker = next_data.get("next_speaker", "NONE")
        
        print(f"    📥 原版判断: {next_speaker}")
        
        if next_speaker == "NONE" or next_speaker not in candidate_characters:
            print(f"原版结束对话，next_speaker={next_speaker}")
            break

        # 生成发言内容（简化版）
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
以及对话历史：{history}
继续说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        response = generate_response(prompt_reply)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": next_speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{next_speaker}: {spoken_line}\n"
    
    print(f"  ✅ 原版生成了{len(dialogue_list)}条对话")
    return dialogue_list

In [4]:
# Cell 3: 改进版本1 - 双重判断法
def generate_dialogue_for_insertion_v1(sentence_context, candidate_characters, full_plot, character_personality):
    """改进版本1：双重判断法"""
    print(f"\n🔍 改进版1：开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""
    MAX_ROUNDS = 10  # 稍微提高上限

    # 第一步：分析对话目标
    goal_prompt = [{
        "role": "system",
        "content": f"""你是故事编剧。分析以下剧情片段需要用对话表达什么：

剧情：{sentence_context}
可用角色：{candidate_characters}

请分析：这段剧情最需要通过对话表达什么？预期几轮对话比较合适？

格式：{{"goal": "对话目标描述", "expected_rounds": 数字}}"""
    }]
    
    goal_response = generate_response(goal_prompt)
    goal_data = convert_json(goal_response)
    dialogue_goal = goal_data.get("goal", "推进剧情")
    expected_rounds = goal_data.get("expected_rounds", 3)
    
    print(f"  🎯 对话目标: {dialogue_goal}")
    print(f"  📊 预期轮数: {expected_rounds}")

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，请基于以下剧情做出第一句发言。
剧情背景：{sentence_context}
对话目标：{dialogue_goal}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue": "...", "action": "..."}}"""
    }]
    
    response = generate_response(prompt_first)
    parsed = convert_json(response)
    
    if isinstance(parsed, dict) and "dialogue" in parsed:
        spoken_line = parsed.get("dialogue", "")
        action = parsed.get("action", "")
        dialogue_list.append({
            "speaker": speaker,
            "dialogue": spoken_line,
            "action": action
        })
        history += f"{speaker}: {spoken_line}\n"

    # 改进的多轮对话循环
    round_count = 0
    while round_count < MAX_ROUNDS:
        round_count += 1
        print(f"\n  🔄 改进版1第{round_count}轮对话")
        
        # 🎯 改进的判断逻辑：双重判断
        judge_prompt = [{
            "role": "system",
            "content": f"""你是故事编剧。请分析当前对话状态：

【剧情背景】：{sentence_context}
【对话目标】：{dialogue_goal}
【预期轮数】：{expected_rounds}
【当前轮数】：{round_count}
【对话历史】：
{history}

请回答两个问题：
1. 对话目标是否已经达成？(0-10分评分)
2. 如果未充分达成，下一个发言人是谁？

可选角色：{candidate_characters}

格式：{{"goal_achieved": 分数, "should_continue": true/false, "next_speaker": "角色名或NONE", "reason": "判断理由"}}"""
        }]
        
        judge_res = generate_response(judge_prompt)
        judge_data = convert_json(judge_res)
        
        goal_achieved = judge_data.get("goal_achieved", 5)
        should_continue = judge_data.get("should_continue", True)
        next_speaker = judge_data.get("next_speaker", "NONE")
        reason = judge_data.get("reason", "")
        
        print(f"    📊 目标达成度: {goal_achieved}/10")
        print(f"    🤔 是否继续: {should_continue}")
        print(f"    👤 下一发言人: {next_speaker}")
        print(f"    💭 理由: {reason}")
        
        # 🎯 改进的停止条件
        if (not should_continue or 
            goal_achieved >= 7 or  # 目标达成度高
            next_speaker == "NONE" or 
            next_speaker not in candidate_characters or
            round_count >= expected_rounds + 2):  # 超过预期轮数
            print(f"改进版1结束对话: goal_achieved={goal_achieved}, should_continue={should_continue}")
            break

        # 生成发言内容
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
对话目标：{dialogue_goal}
对话历史：{history}
继续说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        response = generate_response(prompt_reply)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": next_speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{next_speaker}: {spoken_line}\n"
    
    print(f"  ✅ 改进版1生成了{len(dialogue_list)}条对话")
    return dialogue_list

In [5]:
# Cell 4: 改进版本2 - 简化判断法
def generate_dialogue_for_insertion_v2(sentence_context, candidate_characters, full_plot, character_personality):
    """改进版本2：简化但更明确的判断"""
    print(f"\n🔍 改进版2：开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，基于剧情做出第一句发言：{sentence_context}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue": "...", "action": "..."}}"""
    }]
    
    response = generate_response(prompt_first)
    parsed = convert_json(response)
    
    if isinstance(parsed, dict) and "dialogue" in parsed:
        spoken_line = parsed.get("dialogue", "")
        action = parsed.get("action", "")
        dialogue_list.append({
            "speaker": speaker,
            "dialogue": spoken_line,
            "action": action
        })
        history += f"{speaker}: {spoken_line}\n"

    # 🎯 极简判断循环
    round_count = 0
    while round_count < 6:  # 硬限制
        round_count += 1
        print(f"\n  🔄 改进版2第{round_count}轮对话")
        
        # 🎯 简化但明确的判断
        judge_prompt = [{
            "role": "system",
            "content": f"""剧情：{sentence_context}
已有对话：{history}

这段对话是否已经足够表达剧情内容了？
- 如果已经足够，返回 "STOP"
- 如果还需要继续，从以下角色选择下一个发言人：{candidate_characters}

只返回角色名或"STOP"，不要解释。"""
        }]
        
        judge_res = generate_response(judge_prompt)
        next_speaker = judge_res.strip().strip('"').strip("'")
        
        print(f"    📥 简化判断: {next_speaker}")
        
        if (next_speaker == "STOP" or 
            next_speaker not in candidate_characters or
            len(history.split('\n')) >= 6):  # 额外保护
            print(f"改进版2结束对话: {next_speaker}")
            break

        # 生成发言
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
和对话历史：{history}
说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        response = generate_response(prompt_reply)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": next_speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{next_speaker}: {spoken_line}\n"
    
    print(f"  ✅ 改进版2生成了{len(dialogue_list)}条对话")
    return dialogue_list

In [6]:
# Cell 5: 运行对比测试
print("=" * 80)
print("🧪 开始对比测试三种方法")
print("=" * 80)

# 测试原版
print("\n📍 测试原版方法...")
original_result = generate_dialogue_for_insertion_original(
    test_sentence_context, test_candidates, [], test_characters
)

print(f"\n📊 原版结果：{len(original_result)}轮对话")
for i, d in enumerate(original_result):
    print(f"  {i+1}. {d['speaker']}: {d['dialogue'][:50]}...")

# 测试改进版1
print("\n📍 测试改进版1（双重判断法）...")
v1_result = generate_dialogue_for_insertion_v1(
    test_sentence_context, test_candidates, [], test_characters
)

print(f"\n📊 改进版1结果：{len(v1_result)}轮对话")
for i, d in enumerate(v1_result):
    print(f"  {i+1}. {d['speaker']}: {d['dialogue'][:50]}...")

# 测试改进版2
print("\n📍 测试改进版2（简化判断法）...")
v2_result = generate_dialogue_for_insertion_v2(
    test_sentence_context, test_candidates, [], test_characters
)

print(f"\n📊 改进版2结果：{len(v2_result)}轮对话")
for i, d in enumerate(v2_result):
    print(f"  {i+1}. {d['speaker']}: {d['dialogue'][:50]}...")

🧪 开始对比测试三种方法

📍 测试原版方法...

🔍 原版：开始生成对话，候选角色: ['小红帽', '赛博狼']
原始 content: {"dialogue": "赛博狼，你听到了吗？星际森林好像在说话，机械残骸也在诉说着过去的故事。我们真的到达了一个全新的世界。", "action": "小红帽环顾四周，警觉地走下飞船舱门，试图辨认周围的声音和环境。"}...

  🔄 原版第1轮对话
原始 content: {"next_speaker": "赛博狼"}...
    📥 原版判断: 赛博狼
原始 content: {"dialogue": "小红帽，我的传感器检测到这里的能量波动异常，森林的低语中夹杂着未知的数据流，或许正是这个世界的语言。我们要小心前行，解读它隐藏的信息。", "action": "赛博狼调整耳部天线，启动环境分析模块，缓步走向星际森林的边缘。"}...

  🔄 原版第2轮对话
原始 content: {"next_speaker": "小红帽"}...
    📥 原版判断: 小红帽
原始 content: {"dialogue": "赛博狼，我们可以尝试同步你的数据解析器，让我也能听懂这些星际低语。也许森林会告诉我们如何继续前进。", "action": "小红帽轻轻触摸赛博狼的装甲接口，准备连接自己的通讯腕带，屏幕上开始浮现奇异的能量符号。"}...

  🔄 原版第3轮对话
原始 content: {"next_speaker": "赛博狼"}...
    📥 原版判断: 赛博狼
原始 content: {"dialogue": "小红帽，我正在将数据流译码并同步到你的感知装置，准备好接受信息输入。每一道低语都可能是藏宝图的线索，也可能是危机的警告。你感受到第一串星际密语了吗？", "action": "赛博狼连接小红帽头盔的数据端口，启动同步程序，星际低语化为可视化符号在小红帽的视野中浮现。"}...

  🔄 原版第4轮对话
原始 content: {"next_speaker": "小红帽"}...
    📥 原版判断: 小红帽
原始 content: {"dialogue": "赛博狼，我刚刚在脑海里听到了一句奇怪的星际密语：‘银色树影下，遗忘者的记忆指引方向。’你能定位那个‘银色树影’吗？", "action"

In [7]:
# Cell 6: 结果分析和选择
def analyze_results(original, v1, v2):
    """分析三种方法的效果"""
    print("\n📊 结果对比分析:")
    print(f"原版轮数: {len(original)}")
    print(f"改进版1轮数: {len(v1)}")
    print(f"改进版2轮数: {len(v2)}")
    
    print(f"\n🎯 推荐：")
    if len(v1) < len(original) and len(v1) > 0:
        print("✅ 改进版1效果最好 - 有目标导向且控制了长度")
    elif len(v2) < len(original) and len(v2) > 0:
        print("✅ 改进版2效果不错 - 简单但有效")
    else:
        print("⚠️ 需要进一步调整")

analyze_results(original_result, v1_result, v2_result)

# 选择最佳版本
best_version = None
if len(v1_result) <= 5 and len(v1_result) > 0:
    best_version = "v1"
    best_function = generate_dialogue_for_insertion_v1
elif len(v2_result) <= 5 and len(v2_result) > 0:
    best_version = "v2"
    best_function = generate_dialogue_for_insertion_v2
else:
    best_version = "original"
    best_function = generate_dialogue_for_insertion_original

print(f"\n🏆 最佳版本: {best_version}")


📊 结果对比分析:
原版轮数: 6
改进版1轮数: 2
改进版2轮数: 5

🎯 推荐：
✅ 改进版1效果最好 - 有目标导向且控制了长度

🏆 最佳版本: v1


In [8]:
# Cell 7: 生成最终可替换的函数
def generate_final_function():
    """生成最终可以替换到主程序的函数"""
    
    # 根据测试结果选择最佳版本
    if best_version == "v1":
        function_code = '''
def generate_dialogue_for_insertion(sentence_context, candidate_characters, full_plot, character_personality):
    """
    改进版：双重判断法，带目标导向的对话生成
    """
    print(f"\\n🔍 开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""

    # 第一步：分析对话目标
    goal_prompt = [{
        "role": "system",
        "content": f"""你是故事编剧。分析以下剧情片段需要用对话表达什么：

剧情：{sentence_context}
可用角色：{candidate_characters}

请分析：这段剧情最需要通过对话表达什么？预期几轮对话比较合适？

格式：{{"goal": "对话目标描述", "expected_rounds": 数字}}"""
    }]
    
    goal_response = generate_response(goal_prompt)
    goal_data = convert_json(goal_response)
    dialogue_goal = goal_data.get("goal", "推进剧情")
    expected_rounds = goal_data.get("expected_rounds", 3)

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，请基于以下剧情做出第一句发言。
剧情背景：{sentence_context}
对话目标：{dialogue_goal}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue": "...", "action": "..."}}"""
    }]
    
    response = generate_response(prompt_first)
    parsed = convert_json(response)
    
    if isinstance(parsed, dict) and "dialogue" in parsed:
        spoken_line = parsed.get("dialogue", "")
        action = parsed.get("action", "")
        dialogue_list.append({
            "speaker": speaker,
            "dialogue": spoken_line,
            "action": action
        })
        history += f"{speaker}: {spoken_line}\\n"

    # 多轮对话循环
    round_count = 0
    MAX_ROUNDS = 8
    while round_count < MAX_ROUNDS:
        round_count += 1
        
        # 双重判断逻辑
        judge_prompt = [{
            "role": "system",
            "content": f"""你是故事编剧。请分析当前对话状态：

【剧情背景】：{sentence_context}
【对话目标】：{dialogue_goal}
【预期轮数】：{expected_rounds}
【当前轮数】：{round_count}
【对话历史】：
{history}

请回答两个问题：
1. 对话目标是否已经达成？(0-10分评分)
2. 如果未充分达成，下一个发言人是谁？

可选角色：{candidate_characters}

格式：{{"goal_achieved": 分数, "should_continue": true/false, "next_speaker": "角色名或NONE", "reason": "判断理由"}}"""
        }]
        
        judge_res = generate_response(judge_prompt)
        judge_data = convert_json(judge_res)
        
        goal_achieved = judge_data.get("goal_achieved", 5)
        should_continue = judge_data.get("should_continue", True)
        next_speaker = judge_data.get("next_speaker", "NONE")
        
        # 停止条件
        if (not should_continue or 
            goal_achieved >= 7 or
            next_speaker == "NONE" or 
            next_speaker not in candidate_characters or
            round_count >= expected_rounds + 2):
            break

        # 生成发言内容
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
对话目标：{dialogue_goal}
对话历史：{history}
继续说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        response = generate_response(prompt_reply)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": next_speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{next_speaker}: {spoken_line}\\n"
    
    print(f"  ✅ 生成了{len(dialogue_list)}条对话")
    return dialogue_list
'''
    elif best_version == "v2":
        function_code = '''
def generate_dialogue_for_insertion(sentence_context, candidate_characters, full_plot, character_personality):
    """
    改进版：简化判断法
    """
    print(f"\\n🔍 开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，基于剧情做出第一句发言：{sentence_context}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue": "...", "action": "..."}}"""
    }]
    
    response = generate_response(prompt_first)
    parsed = convert_json(response)
    
    if isinstance(parsed, dict) and "dialogue" in parsed:
        spoken_line = parsed.get("dialogue", "")
        action = parsed.get("action", "")
        dialogue_list.append({
            "speaker": speaker,
            "dialogue": spoken_line,
            "action": action
        })
        history += f"{speaker}: {spoken_line}\\n"

    # 简化判断循环
    round_count = 0
    while round_count < 6:
        round_count += 1
        
        # 简化判断
        judge_prompt = [{
            "role": "system",
            "content": f"""剧情：{sentence_context}
已有对话：{history}

这段对话是否已经足够表达剧情内容了？
- 如果已经足够，返回 "STOP"
- 如果还需要继续，从以下角色选择下一个发言人：{candidate_characters}

只返回角色名或"STOP"，不要解释。"""
        }]
        
        judge_res = generate_response(judge_prompt)
        next_speaker = judge_res.strip().strip('"').strip("'")
        
        if (next_speaker == "STOP" or 
            next_speaker not in candidate_characters or
            len(history.split('\\n')) >= 6):
            break

        # 生成发言
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
和对话历史：{history}
说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        response = generate_response(prompt_reply)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": next_speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{next_speaker}: {spoken_line}\\n"
    
    print(f"  ✅ 生成了{len(dialogue_list)}条对话")
    return dialogue_list
'''
    else:
        function_code = "# 使用原版函数"
    
    print("🎯 最终可替换的函数代码：")
    print("=" * 60)
    print(function_code)
    print("=" * 60)
    
    return function_code

final_code = generate_final_function()

🎯 最终可替换的函数代码：

def generate_dialogue_for_insertion(sentence_context, candidate_characters, full_plot, character_personality):
    """
    改进版：双重判断法，带目标导向的对话生成
    """
    print(f"\n🔍 开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""

    # 第一步：分析对话目标
    goal_prompt = [{
        "role": "system",
        "content": f"""你是故事编剧。分析以下剧情片段需要用对话表达什么：

剧情：{sentence_context}
可用角色：{candidate_characters}

请分析：这段剧情最需要通过对话表达什么？预期几轮对话比较合适？

格式：{{"goal": "对话目标描述", "expected_rounds": 数字}}"""
    }]
    
    goal_response = generate_response(goal_prompt)
    goal_data = convert_json(goal_response)
    dialogue_goal = goal_data.get("goal", "推进剧情")
    expected_rounds = goal_data.get("expected_rounds", 3)

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，请基于以下剧情做出第一句发言。
剧情背景：{sentence_context}
对话目标：{dialogue_goal}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue":

✅ 成功导入项目模块
📁 使用输出目录: /Users/haha/Story/data/output/小红帽_科幻_nonlinear_T0.3_S3
📚 加载了story.json: 6 个章节
👥 加载了characters.json: 9 个角色
  - 小红帽: 无描述
  - 祖母: 无描述
  - 机械大灰狼: 无描述
  - 银河巡逻队队长: 无描述
  - 银河巡逻队队员甲: 无描述
  - 银河巡逻队队员乙: 无描述
  - 智能助手（小智）: 无描述
  - 虚拟野兽: 无描述
  - 未知幕后主使: 无描述

✅ 数据加载完成


In [10]:
# Cell 1: 环境设置和数据加载（更新版）
import json
import sys
import os
from pathlib import Path

# 设置项目路径
project_path = "/Users/haha/Story"
sys.path.append(project_path)

# 导入必要函数
try:
    from src.utils.utils import generate_response, convert_json, split_plot_into_sentences
    print("✅ 成功导入项目模块")
except ImportError as e:
    print(f"⚠️ 导入失败: {e}")
    # 如果导入失败，创建模拟函数
    def generate_response(messages):
        return '{"next_speaker": "NONE"}'
    def convert_json(text):
        try:
            return json.loads(text)
        except:
            return {}
    def split_plot_into_sentences(text):
        return text.split('。')

# 🔍 加载你的半成品数据
output_dir = Path("/Users/haha/Story/data/output/小红帽_科幻_nonlinear_T0.3_S3")

print(f"📁 使用输出目录: {output_dir}")

# 加载story.json
story_file = output_dir / "story.json"
if story_file.exists():
    with open(story_file, 'r', encoding='utf-8') as f:
        real_story = json.load(f)
    print(f"📚 加载了story.json: {len(real_story)} 个章节")
else:
    print("❌ 未找到story.json")
    real_story = []

# 加载characters.json（处理你的复杂格式）
characters_file = output_dir / "characters.json"
if characters_file.exists():
    with open(characters_file, 'r', encoding='utf-8') as f:
        real_characters = json.load(f)
    print(f"👥 加载了characters.json: {len(real_characters)} 个角色")
    
    # 展示角色信息
    for char in real_characters:
        name = char.get('name', '未知')
        role = char.get('role', '未知角色')
        traits = char.get('traits', '无描述')
        print(f"  - {name}: {role}")
        print(f"    特点: {traits[:50]}...")
    
    # 🎯 提取角色名列表，供对话生成使用
    character_names = [char.get('name', '') for char in real_characters if char.get('name')]
    print(f"\n📝 提取的角色名列表: {character_names}")
    
else:
    print("❌ 未找到characters.json")
    real_characters = []
    character_names = []

print("\n✅ 数据加载完成")

✅ 成功导入项目模块
📁 使用输出目录: /Users/haha/Story/data/output/小红帽_科幻_nonlinear_T0.3_S3
📚 加载了story.json: 6 个章节
👥 加载了characters.json: 9 个角色
  - 小红帽: 主角,星际快递员/冒险者
    特点: 机敏、勇敢、善良、富有同情心、坚韧不拔...
  - 祖母: 小红帽的外星亲人,任务目标
    特点: 慈爱、睿智、体弱多病、乐观、关怀家人...
  - 机械大灰狼: 主要反派,对手,智能机械体
    特点: 狡猾、计算精准、善于伪装、缺乏情感、执着于任务...
  - 银河巡逻队队长: 银河巡逻队领导,救援者
    特点: 果断、公正、富有责任心、经验丰富...
  - 银河巡逻队队员甲: 银河巡逻队成员,救援者
    特点: 冷静、技术娴熟、配合默契...
  - 银河巡逻队队员乙: 银河巡逻队成员,救援者
    特点: 勇敢、反应迅速、富有正义感...
  - 智能助手（小智）: 小红帽的AI伙伴,辅助者
    特点: 聪明、忠诚、幽默、学习能力强、善于分析...
  - 虚拟野兽: 量子森林中的障碍/对手,环境威胁
    特点: 不可预测、凶猛、数据化、具备攻击性...
  - 未知幕后主使: 机械大灰狼的控制者/黑暗势力
    特点: 神秘、狡诈、权力欲强、隐匿行事...

📝 提取的角色名列表: ['小红帽', '祖母', '机械大灰狼', '银河巡逻队队长', '银河巡逻队队员甲', '银河巡逻队队员乙', '智能助手（小智）', '虚拟野兽', '未知幕后主使']

✅ 数据加载完成


In [11]:
# Cell 2: 智能角色选择函数
def select_relevant_characters(sentence, all_character_names, max_chars=3):
    """
    根据句子内容智能选择相关角色
    """
    relevant_chars = []
    
    # 先找句子中直接提到的角色
    for name in all_character_names:
        if name in sentence:
            relevant_chars.append(name)
    
    # 如果没有直接提到角色，根据关键词推断
    if not relevant_chars:
        if any(word in sentence for word in ['快递', '运送', '任务', '芯片']):
            if '小红帽' in all_character_names:
                relevant_chars.append('小红帽')
        
        if any(word in sentence for word in ['祖母', '奶奶', '老人']):
            if '祖母' in all_character_names:
                relevant_chars.append('祖母')
        
        if any(word in sentence for word in ['机械', '狼', '敌人', '攻击']):
            if '机械大灰狼' in all_character_names:
                relevant_chars.append('机械大灰狼')
        
        if any(word in sentence for word in ['助手', '智能', 'AI']):
            if '智能助手（小智）' in all_character_names:
                relevant_chars.append('智能助手（小智）')
    
    # 如果还是没有，默认选择主要角色
    if not relevant_chars:
        main_chars = ['小红帽', '祖母', '机械大灰狼']
        for char in main_chars:
            if char in all_character_names:
                relevant_chars.append(char)
                if len(relevant_chars) >= 2:
                    break
    
    # 限制角色数量
    return relevant_chars[:max_chars]

# 测试角色选择
test_sentence = "小红帽驾驶着快递飞船，准备将芯片送到祖母那里"
selected = select_relevant_characters(test_sentence, character_names)
print(f"🧪 测试句子: {test_sentence}")
print(f"🎯 选择的角色: {selected}")

🧪 测试句子: 小红帽驾驶着快递飞船，准备将芯片送到祖母那里
🎯 选择的角色: ['小红帽', '祖母']


In [12]:
# Cell 3: 从真实数据提取测试用例（更新版）
def extract_test_cases_from_story_updated(story_data, character_names, max_cases=3):
    """从你的real story中提取测试用例 - 处理复杂格式"""
    test_cases = []
    
    for i, chapter in enumerate(story_data[:max_cases]):
        plot = chapter.get("plot", "")
        chapter_characters = chapter.get("characters", [])
        
        if plot:
            # 取plot的前两句作为测试
            sentences = split_plot_into_sentences(plot)
            if len(sentences) >= 2:
                test_sentence = sentences[0] + "。" + sentences[1] + "。"
            else:
                test_sentence = plot[:200] + "。"  # 如果句子不够，取前200字符
            
            # 🎯 智能选择相关角色
            relevant_characters = select_relevant_characters(test_sentence, character_names, max_chars=3)
            
            # 如果智能选择失败，尝试从章节中提取
            if not relevant_characters and chapter_characters:
                for char_info in chapter_characters:
                    if isinstance(char_info, dict):
                        name = char_info.get("name", "")
                        if name and name in character_names:
                            relevant_characters.append(name)
                    elif isinstance(char_info, str) and char_info in character_names:
                        relevant_characters.append(char_info)
                
                relevant_characters = relevant_characters[:3]
            
            # 确保至少有2个角色
            if len(relevant_characters) >= 2:
                test_cases.append({
                    "chapter_id": chapter.get("chapter_id", f"Chapter_{i+1}"),
                    "title": chapter.get("title", "未知标题"),
                    "sentence": test_sentence,
                    "characters": relevant_characters,
                    "scene": chapter.get("scene", ""),
                    "original_plot": plot[:100] + "..."  # 保存原始plot片段用于参考
                })
    
    return test_cases

# 提取测试用例
if real_story and character_names:
    test_cases = extract_test_cases_from_story_updated(real_story, character_names, max_cases=3)
    print(f"🧪 从你的story.json中提取了 {len(test_cases)} 个测试用例:")
    for i, case in enumerate(test_cases):
        print(f"\n  {i+1}. {case['chapter_id']}: {case['title']}")
        print(f"     测试句子: {case['sentence'][:80]}...")
        print(f"     选择角色: {case['characters']}")
        print(f"     场景: {case['scene'][:60]}...")
else:
    print("❌ 没有story数据或角色数据，无法提取测试用例")
    test_cases = []

🧪 从你的story.json中提取了 1 个测试用例:

  1. Chapter 1: 星际小红帽的任务
     测试句子: 小红帽紧盯着导航终端上的航线,内心充满着焦虑与希望.。她的思绪时常被回忆牵引,浮现出祖母慈祥的面容——在偏远太空站里,那位年迈却乐观的亲人,曾用颤抖的手抚摸她的...
     选择角色: ['小红帽', '祖母']
     场景: 星际快递飞船“赤焰号”内,小红帽独自驾驶着飞船穿梭于点缀着星云与小行星的幽暗宇宙航道.飞船舱壁上映出明灭的仪表灯,外部舷...


In [14]:
# Cell 4: 定义改进版对话生成函数（适配你的角色格式）
def generate_dialogue_improved_for_your_data(sentence_context, candidate_characters, full_plot, character_personality):
    """
    改进版对话生成 - 专门适配你的角色数据格式
    """
    print(f"\n🔍 改进版：开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""

    # 🎯 利用你的详细角色信息
    character_info = {}
    for char in character_personality:
        name = char.get('name', '')
        if name in candidate_characters:
            character_info[name] = {
                'role': char.get('role', ''),
                'traits': char.get('traits', ''),
                'background': char.get('background', ''),
                'motivation': char.get('motivation', '')
            }
    
    print(f"  📋 角色信息: {list(character_info.keys())}")

    # 分析对话目标 - 结合角色背景
    goal_prompt = [{
        "role": "system",
        "content": f"""你是故事编剧。分析以下剧情片段需要用对话表达什么：

剧情：{sentence_context}
可用角色：{candidate_characters}

角色背景信息：
{json.dumps(character_info, ensure_ascii=False, indent=2)}

请分析：这段剧情最需要通过对话表达什么？预期几轮对话比较合适？

⚠️ 重要：expected_rounds必须是单个整数（如2、3、4），不能是范围

格式：{{"goal": "对话目标描述", "expected_rounds": 整数}}"""
    }]
    
    try:
        goal_response = generate_response(goal_prompt)
        goal_data = convert_json(goal_response)
        
        if isinstance(goal_data, dict):
            dialogue_goal = goal_data.get("goal", "推进剧情")
            expected_rounds = goal_data.get("expected_rounds", 3)
            
            try:
                expected_rounds = int(expected_rounds)
            except (ValueError, TypeError):
                expected_rounds = 3
        else:
            dialogue_goal = "推进剧情"
            expected_rounds = 3
            
    except Exception as e:
        print(f"⚠️ 目标分析失败: {e}")
        dialogue_goal = "推进剧情"
        expected_rounds = 3

    print(f"  🎯 对话目标: {dialogue_goal}")
    print(f"  📊 预期轮数: {expected_rounds}")

    # 第一个发言人 - 结合角色性格
    speaker = candidate_characters[0]
    speaker_info = character_info.get(speaker, {})
    
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}。

角色信息：
- 角色定位：{speaker_info.get('role', '未知')}
- 性格特点：{speaker_info.get('traits', '未知')}
- 背景：{speaker_info.get('background', '未知')}
- 动机：{speaker_info.get('motivation', '未知')}

请基于以下剧情做出第一句发言：
剧情背景：{sentence_context}
对话目标：{dialogue_goal}
其他角色：{[c for c in candidate_characters if c != speaker]}

请说出符合你性格特点和动机的话，格式：
{{"dialogue": "...", "action": "..."}}"""
    }]
    
    try:
        response = generate_response(prompt_first)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{speaker}: {spoken_line}\n"
    except Exception as e:
        print(f"⚠️ 第一句对话生成失败: {e}")
        return []

    # 多轮对话循环
    round_count = 0
    SAFETY_LIMIT = 10
    
    while round_count < SAFETY_LIMIT:
        round_count += 1
        print(f"\n  🔄 改进版第{round_count}轮对话")
        
        # 判断是否继续
        judge_prompt = [{
            "role": "system",
            "content": f"""你是故事编剧。请分析当前对话状态：

【剧情背景】：{sentence_context}
【对话目标】：{dialogue_goal}
【预期轮数】：{expected_rounds}
【当前轮数】：{round_count}
【对话历史】：
{history}

【角色信息】：
{json.dumps(character_info, ensure_ascii=False, indent=2)}

请判断：
1. 对话目标是否已经达成？(0-10分评分)
2. 是否应该继续对话？
3. 如果继续，哪个角色最适合说下一句？

可选角色：{candidate_characters}

格式：{{"goal_achieved": 分数, "should_continue": true/false, "next_speaker": "角色名或NONE", "reason": "判断理由"}}"""
        }]
        
        try:
            judge_res = generate_response(judge_prompt)
            judge_data = convert_json(judge_res)
            
            if not isinstance(judge_data, dict):
                print(f"⚠️ 判断返回非dict格式")
                break
                
            goal_achieved = judge_data.get("goal_achieved", 5)
            should_continue = judge_data.get("should_continue", False)
            next_speaker = judge_data.get("next_speaker", "NONE")
            reason = judge_data.get("reason", "")
            
            print(f"    📊 目标达成度: {goal_achieved}/10")
            print(f"    🤔 是否继续: {should_continue}")
            print(f"    👤 下一发言人: {next_speaker}")
                
        except Exception as e:
            print(f"⚠️ 对话判断失败: {e}")
            break
        
        # 停止条件
        if (not should_continue or
            next_speaker == "NONE" or 
            next_speaker not in candidate_characters):
            print(f"  🛑 改进版结束对话")
            break

        # 生成下一句对话
        next_speaker_info = character_info.get(next_speaker, {})
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}。

角色信息：
- 角色定位：{next_speaker_info.get('role', '未知')}
- 性格特点：{next_speaker_info.get('traits', '未知')}
- 背景：{next_speaker_info.get('background', '未知')}
- 动机：{next_speaker_info.get('motivation', '未知')}

基于以下情况说一句话：
剧情：{sentence_context}
对话目标：{dialogue_goal}
对话历史：{history}

请说出符合你性格和动机的话，格式：
{{"dialogue": "...", "action": "..."}}"""
        }]
        
        try:
            response = generate_response(prompt_reply)
            parsed = convert_json(response)
            
            if isinstance(parsed, dict) and "dialogue" in parsed:
                spoken_line = parsed.get("dialogue", "")
                action = parsed.get("action", "")
                dialogue_list.append({
                    "speaker": next_speaker,
                    "dialogue": spoken_line,
                    "action": action
                })
                history += f"{next_speaker}: {spoken_line}\n"
        except Exception as e:
            print(f"⚠️ 发言生成失败: {e}")
            continue
    
    if round_count >= SAFETY_LIMIT:
        print(f"⚠️ 达到安全保底({SAFETY_LIMIT}轮)")
    
    print(f"  ✅ 改进版生成了{len(dialogue_list)}条对话")
    return dialogue_list

In [15]:
# Cell 5: 运行针对你数据的专门测试
def run_your_data_test():
    """专门针对你的数据格式运行测试"""
    
    if not test_cases:
        print("❌ 没有可用的测试用例")
        return []
    
    results = []
    
    for i, case in enumerate(test_cases):
        print(f"\n{'='*80}")
        print(f"🧪 测试用例 {i+1}: {case['chapter_id']}")
        print(f"📖 标题: {case['title']}")
        print(f"📝 测试句子: {case['sentence']}")
        print(f"👥 选择角色: {case['characters']}")
        print(f"🎬 场景: {case['scene']}")
        print('='*80)
        
        # 只测试改进版，因为重点是验证新函数
        print(f"\n📍 测试改进版方法...")
        try:
            improved_result = generate_dialogue_improved_for_your_data(
                case['sentence'], 
                case['characters'], 
                [], 
                real_characters  # 传入完整的角色信息
            )
            print(f"✅ 改进版完成: {len(improved_result)}轮对话")
            
            # 展示生成的对话
            print(f"\n💬 生成的对话:")
            for j, dialogue in enumerate(improved_result):
                print(f"  {j+1}. {dialogue['speaker']}: {dialogue['dialogue']}")
                if dialogue['action']:
                    print(f"     动作: {dialogue['action']}")
            
        except Exception as e:
            print(f"❌ 改进版失败: {e}")
            import traceback
            traceback.print_exc()
            improved_result = []
        
        # 记录结果
        result = {
            "case": case,
            "improved_rounds": len(improved_result),
            "improved_dialogue": improved_result,
            "success": len(improved_result) > 0
        }
        results.append(result)
        
        print(f"\n📊 本轮结果: {len(improved_result)}轮对话")
    
    return results

# 运行你的专门测试
if test_cases:
    print("🚀 开始使用你的真实数据测试改进版对话生成...")
    your_test_results = run_your_data_test()
else:
    print("⚠️ 没有测试用例，跳过测试")
    your_test_results = []

🚀 开始使用你的真实数据测试改进版对话生成...

🧪 测试用例 1: Chapter 1
📖 标题: 星际小红帽的任务
📝 测试句子: 小红帽紧盯着导航终端上的航线,内心充满着焦虑与希望.。她的思绪时常被回忆牵引,浮现出祖母慈祥的面容——在偏远太空站里,那位年迈却乐观的亲人,曾用颤抖的手抚摸她的发梢,鼓励她勇敢追梦.。
👥 选择角色: ['小红帽', '祖母']
🎬 场景: 星际快递飞船“赤焰号”内,小红帽独自驾驶着飞船穿梭于点缀着星云与小行星的幽暗宇宙航道.飞船舱壁上映出明灭的仪表灯,外部舷窗投下深蓝与紫色交织的星河光芒,舱内空间虽狭小却井然有序,一只红色披风随小红帽的动作在座椅边垂下.仪表盘上,医疗纳米芯片安静地封存于全息防护盒中,散发着微微蓝光.

📍 测试改进版方法...

🔍 改进版：开始生成对话，候选角色: ['小红帽', '祖母']
  📋 角色信息: ['小红帽', '祖母']
原始 content: {"goal": "通过对话展现小红帽对祖母的思念、担忧与希望，以及祖母对小红帽的鼓励和爱，强化祖孙间深厚情感，为小红帽冒险动机提供情感铺垫。", "expected_rounds": 2}...
  🎯 对话目标: 通过对话展现小红帽对祖母的思念、担忧与希望,以及祖母对小红帽的鼓励和爱,强化祖孙间深厚情感,为小红帽冒险动机提供情感铺垫.
  📊 预期轮数: 2
原始 content: {"dialogue": "导航说还要两个星系才能到达，但我真的好担心……奶奶，您一定要等我啊，我带着最好的医疗纳米芯片来了！", "action": "小红帽紧紧握住怀里的快递盒，眼中浮现出祖母温柔的笑容，轻声呢喃，仿佛祖母就在身边。"}...

  🔄 改进版第1轮对话
原始 content: {"goal_achieved": 5, "should_continue": true, "next_speaker": "祖母", "reason": "小红帽已经表达了对祖母的思念和担忧，并展现了希望，但祖母的回应尚未出现，祖孙间的深厚情感还未完全展现。需要祖母回应，给予鼓励和爱，进一步强化情感，为小红帽的冒险动机做更充分的铺垫。"}...
    📊 目标达成度: 5/10
    🤔 是否继续: True
    👤 下一发言人: 祖母
原始 content: {

In [17]:
# Cell 6: 分析你的测试结果
def analyze_your_test_results(results):
    """分析你的测试结果"""
    if not results:
        print("❌ 没有测试结果可分析")
        return False
    
    print(f"\n{'='*60}")
    print("📊 你的真实数据测试结果")
    print('='*60)
    
    success_count = sum(1 for r in results if r['success'])
    total_count = len(results)
    average_rounds = sum(r['improved_rounds'] for r in results) / len(results) if results else 0
    
    print(f"📈 基本统计:")
    print(f"   测试用例总数: {total_count}")
    print(f"   成功生成对话: {success_count}")
    print(f"   成功率: {success_count/total_count*100:.1f}%")
    print(f"   平均对话轮数: {average_rounds:.1f}轮")
    
    print(f"\n📝 详细结果:")
    for i, result in enumerate(results):
        case = result["case"]
        print(f"\n{i+1}. {case['chapter_id']}: {case['title']}")
        print(f"   对话轮数: {result['improved_rounds']}")
        print(f"   角色组合: {case['characters']}")
        
        if result["improved_dialogue"]:
            print("   对话示例:")
            for j, d in enumerate(result["improved_dialogue"][:2]):
                print(f"     {j+1}. {d['speaker']}: {d['dialogue'][:60]}...")
    
    # 评估结果
    if average_rounds <= 0:
        quality = "❌ 失败"
        recommendation = "函数存在严重问题，需要调试"
        should_replace = False
    elif average_rounds > 8:
        quality = "⚠️ 过长"
        recommendation = "对话轮数偏多，需要调整判断逻辑"
        should_replace = False
    elif 3 <= average_rounds <= 6:
        quality = "✅ 理想"
        recommendation = "对话长度合理，可以替换到主程序"
        should_replace = True
    elif average_rounds < 3:
        quality = "⚠️ 过短"
        recommendation = "对话可能过于简短，但比无限循环好"
        should_replace = True
    else:
        quality = "👍 良好"
        recommendation = "效果不错，建议替换"
        should_replace = True
    
    print(f"\n🎯 最终评估:")
    print(f"   质量评级: {quality}")
    print(f"   平均轮数: {average_rounds:.1f}轮")
    print(f"   推荐操作: {recommendation}")
    
    return should_replace

# 分析你的结果
if your_test_results:
    should_replace = analyze_your_test_results(your_test_results)
    
    if should_replace:
        print(f"\n🎉 测试通过！可以安全替换到主程序")
        print(f"\n📋 下一步操作:")
        print("1. 将Cell 4中的函数重命名为 generate_dialogue_for_insertion")
        print("2. 复制到 /Users/haha/Story/src/generation/dialogue_inserter.py")
        print("3. 运行完整流程测试")
    else:
        print(f"\n🤔 需要进一步调试和优化")
else:
    print("⚠️ 请先运行测试")


📊 你的真实数据测试结果
📈 基本统计:
   测试用例总数: 1
   成功生成对话: 1
   成功率: 100.0%
   平均对话轮数: 3.0轮

📝 详细结果:

1. Chapter 1: 星际小红帽的任务
   对话轮数: 3
   角色组合: ['小红帽', '祖母']
   对话示例:
     1. 小红帽: 导航说还要两个星系才能到达,但我真的好担心……奶奶,您一定要等我啊,我带着最好的医疗纳米芯片来了！...
     2. 祖母: 傻孩子,别担心奶奶,奶奶一直在等你呢.你能这么勇敢,奶奶已经很骄傲了.旅途漫长,你也要照顾好自己,等你平安到来,奶奶一定...

🎯 最终评估:
   质量评级: ✅ 理想
   平均轮数: 3.0轮
   推荐操作: 对话长度合理，可以替换到主程序

🎉 测试通过！可以安全替换到主程序

📋 下一步操作:
1. 将Cell 4中的函数重命名为 generate_dialogue_for_insertion
2. 复制到 /Users/haha/Story/src/generation/dialogue_inserter.py
3. 运行完整流程测试
